In [2]:
import random
from enum import StrEnum
from threading import ExceptHookArgs
from typing import Self

from rich.console import Console
from rich.table import Table

console = Console()

class Names(StrEnum):
    JOHN = "John Lennon"
    PAUL = "Paul McCartney"
    GEORGE = "George Harrison"
    RINGO = "Ringo Starr"


class Beetle:
    health_points: int
    name: Names

    def __init__(
        self,
        health_points: int = 100,
        max_health_points: int = 100,
        name: Names = Names.JOHN,
    ) -> None:
        self.health_points = health_points
        self.max_health_points = max_health_points
        self.name = name

    def __eq__(self, other: Self) -> bool:
        return self.health_points == other.health_points

    def __lt__(self, other: Self) -> bool:
        return self.health_points < other.health_points

    def __le__(self, other: Self) -> bool:
        return self.health_points <= other.health_points

    def __str__(self) -> str:
        return f'Beetle(name="{self.name}", hp={self.health_points!r})'

    def styling(self) -> str:
        if self.name is Names.JOHN:
            return "in Johny style"
        elif self.name is Names.PAUL:
            return "in McCartney style"
        return "without style"

    def alive(self):
        return self.hp > 0

    def attack(self, other: Self) -> None:
        damage = 10
        other.health_points -= damage
        console.print(f"[red3]{self.name}[/red3] attacks [dodger_blue3]{other.name}[/dodger_blue3] for {damage} damage")
        if other.health_points <= 0:
            other.health_points = 0
            console.print(f"[dark_red]{other.name} died![/dark_red]")
            self.health_points = min(self.max_health_points, self.health_points + 10)
            console.print(f"[gold1]{self.name}[/gold1] rewarded with [gold3] {10} [/gold3] health points")

class BeetlesArmy:
    beetles_list: list[Beetle]
    beetles_name: Names
    beetles_max_health_points: int

    def __init__(
        self,
        beetles_name: Names,
        beetles_army_size: int = 20,
        beetles_max_health_points: int = 100,
    ):
        self.beetles_list = []
        self.beetles_name = beetles_name
        self.beetles_max_health_points = beetles_max_health_points

        for _ in range(beetles_army_size):
            beetle = Beetle(
                health_points=random.randint(1, self.beetles_max_health_points),
                name=self.beetles_name,
            )
            self.beetles_list.append(beetle)

    def __len__(self) -> int:
        return len(self.beetles_list)

    def __add__(self, other: Self) -> Self:
        if self.beetles_name != other.beetles_name:
            raise ValueError("[turquoise4]Cannot make two different-named beetles friends[/turquoise4]")
        new_beetles_list: list[Beetle] = self.beetles_list + other.beetles_list
        new_army = self.__class__(
            beetles_army_size=1,
            beetles_name=self.beetles_name,
            beetles_max_health_points=self.beetles_max_health_points,
        )
        new_army.beetles_list = new_beetles_list
        return new_army

    def print_army_listing(self):
        for beetle in self.beetles_list:
            print(beetle)

    def alive_beetles(self):
        return [beetle for beetle in self.beetles_list if beetle.health_points > 0] #возвращает список со всеми выжившими

    def is_defeated(self) -> bool:
        return len(self.alive_beetles()) == 0

    def attack(self, other: Self):
        self.beetles_list[0].attack(other.beetles_list[0])

        if other.beetles_list[0].health_points <= 0:
            other.beetles_list[0].health_points = 0
            self.beetles_list[0].health_points += 10  # reward
            other.beetles_list.pop(0)

        other.beetles_list[0].attack(self.beetles_list[0])

        if self.beetles_list[0].health_points <= 0:
            self.beetles_list[0].health_points = 0
            other.beetles_list.pop(0) #убираем


    def battle(self, other: Self):
        round_num = 1
        while self.alive_beetles() and other.alive_beetles():
            console.print(f"\n [orange3]--- ROUND {round_num} ---[/orange3]")
            self.attack(other)
            round_num += 1

        if self.total_health() > other.total_health():
            console.print(f"\n[chartreuse3]{self.beetles_name.value} wins![/chartreuse3]")
        elif self.total_health() < other.total_health():
            console.print(f"\n[chartreuse3]{other.beetles_name.value} wins![/chartreuse3]")
        else:
            console.print("\n[magenta]It's a tie![/magenta]")

    def total_health(self):
        return sum(beetles.health_points for beetles in self.beetles_list)

    # Army comparisons
    def __gt__(self, other: Self):
        return self.total_health() > other.total_health()

    def __lt__(self, other: Self):
        return self.total_health() < other.total_health()

    def __eq__(self, other: Self):
        return self.total_health() == other.total_health()

if __name__ == "__main__":
    warrior_1 = Names[input(f"Enter first army name (JOHN/PAUL/GEORGE/RINGO):").upper()]
    num_1 = int(input(f"Enter first army size: "))

    warrior_2 = Names[input(f"Enter second army name (JOHN/PAUL/GEORGE/RINGO):").upper()]
    num_2 = int(input(f"Enter second army size: "))

    if warrior_1 == warrior_2:
        print("Both armies have the same name! Try again.")
        exit()

    army1 = BeetlesArmy(beetles_name=warrior_1, beetles_army_size=num_1)
    army2 = BeetlesArmy(beetles_name=warrior_2, beetles_army_size=num_2)

    print('[red1]ARMY 1[/red1]')

    army1.print_army_listing()
    print('')

    print('[slate_blue1]ARMY 2[/slate_blue1]')
    army1.print_army_listing()
    print('')

    army1.battle(army2)

Enter first army name (JOHN/PAUL/GEORGE/RINGO):JOHN
Enter first army size: 40
Enter second army name (JOHN/PAUL/GEORGE/RINGO):RINGO
Enter second army size: 30
[red1]ARMY 1[/red1]
Beetle(name="John Lennon", hp=17)
Beetle(name="John Lennon", hp=50)
Beetle(name="John Lennon", hp=20)
Beetle(name="John Lennon", hp=77)
Beetle(name="John Lennon", hp=61)
Beetle(name="John Lennon", hp=46)
Beetle(name="John Lennon", hp=33)
Beetle(name="John Lennon", hp=18)
Beetle(name="John Lennon", hp=77)
Beetle(name="John Lennon", hp=56)
Beetle(name="John Lennon", hp=83)
Beetle(name="John Lennon", hp=24)
Beetle(name="John Lennon", hp=3)
Beetle(name="John Lennon", hp=67)
Beetle(name="John Lennon", hp=81)
Beetle(name="John Lennon", hp=65)
Beetle(name="John Lennon", hp=4)
Beetle(name="John Lennon", hp=83)
Beetle(name="John Lennon", hp=47)
Beetle(name="John Lennon", hp=13)
Beetle(name="John Lennon", hp=38)
Beetle(name="John Lennon", hp=15)
Beetle(name="John Lennon", hp=74)
Beetle(name="John Lennon", hp=75)
Beetle(

--- ROUND 1 ---

John Lennon attacks Ringo Starr for 10 damage

Ringo Starr attacks John Lennon for 10 damage

--- ROUND 2 ---

...

John Lennon attacks Ringo Starr for 10 damage

Ringo Starr attacks John Lennon for 10 damage

John Lennon died!

Ringo Starr rewarded with  10  health points

John Lennon wins!